# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [80]:
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [81]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [83]:
import os
from glob import glob
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files = [x for x in parquet_files if os.path.isfile(x)]



In [84]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532242,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206844,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712807,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276848,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299927,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-06-28,176.460007,177.070007,173.029999,173.360001,173.360001,4189600,Health Care,Pharmaceuticals,2024
ZTS,2024-07-01,173.259995,175.460007,172.000000,172.580002,172.580002,1589800,Health Care,Pharmaceuticals,2024
ZTS,2024-07-02,172.490005,175.990005,172.490005,174.809998,174.809998,2454200,Health Care,Pharmaceuticals,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [85]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False
    ).apply(lambda x: x.assign(Close_lag = x['Close'].shift(1))))

dd_feat = (dd_feat.groupby('ticker', group_keys=False
     ).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1))
     ).assign(returns = lambda x: x['Adj Close']/x['Adj_Close_lag']-1
     ).assign(hi_lo_range = lambda x: x['High']-x['Low']))

/var/folders/9j/x9k9k99508b1c0fbb8n5vj1m0000gp/T/ipykernel_1807/572876171.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False
/var/folders/9j/x9k9k99508b1c0fbb8n5vj1m0000gp/T/ipykernel_1807/572876171.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat.groupby('ticker', group_keys=False


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [86]:
df = dd_feat.compute()
df.groupby('ticker', group_keys=False).apply(lambda x: x.assign(roll_avg_ret = x['returns'].rolling(10).mean()))


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,roll_avg_ret
ticker,,,,,,,,,,,,,,,
IP,2003-01-02,32.695671,33.386753,32.266079,33.181293,15.344504,3215136,Materials,Paper & Plastic Packaging Products & Materials,2003,NaN,NaN,NaN,1.120674,NaN
IP,2003-01-03,33.013195,33.181293,32.592941,32.891788,15.210626,2418687,Materials,Paper & Plastic Packaging Products & Materials,2003,33.181293,15.344504,-0.008725,0.588352,NaN
IP,2003-01-06,32.835754,34.021801,32.835754,33.704277,15.586359,2977422,Materials,Paper & Plastic Packaging Products & Materials,2003,32.891788,15.210626,0.024702,1.186047,NaN
IP,2003-01-07,33.648243,34.031139,33.396091,33.881718,15.668419,2864990,Materials,Paper & Plastic Packaging Products & Materials,2003,33.704277,15.586359,0.005265,0.635048,NaN
IP,2003-01-08,33.573532,33.610886,32.966499,33.087906,15.301325,2607466,Materials,Paper & Plastic Packaging Products & Materials,2003,33.881718,15.668419,-0.023429,0.644386,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BXP,2001-12-24,38.000000,38.500000,38.000000,38.320000,13.684837,73700,Real Estate,Office REITs,2001,37.930000,13.545555,0.010283,0.500000,0.001519
BXP,2001-12-26,37.750000,38.150002,37.700001,38.099998,13.815375,292000,Real Estate,Office REITs,2001,38.320000,13.684837,0.009539,0.450001,0.002472
BXP,2001-12-27,38.200001,38.500000,37.990002,37.990002,13.775488,126600,Real Estate,Office REITs,2001,38.099998,13.815375,-0.002887,0.509998,0.002449


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.